# Analiza wypływu parametrów na algorytm mrówkowy

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import re
from pathlib import Path
from glob import glob
from collections import defaultdict

Przydatne funkcje

In [ ]:
def load_mrowczenie(file_path: str) -> defaultdict:
    line_pattern = r'Pokolenie nr=\s*(?P<generation>\d+)\s+najlepszy=\s*(?P<best_ant_num>\d+)\(dl=\s*(?P<best_value>[-+]?[0-9]*\.?[0-9]*)\),\s*najgorszy=\s*(?P<worst_ant_num>\d+)\(dl=\s*(?P<worst_value>[-+]?[0-9]*\.?[0-9]*)\),\s*średnia_dl=\s*(?P<mean_value>[-+]?[0-9]*\.?[0-9]*)'
    
    with open(file_path, 'r', encoding='windows-1250') as file:
        lines = file.readlines()
        
#     data = []
    data = defaultdict(list)
    for line in lines:
        match = re.match(line_pattern, line)
        if not match:
            print('Line didn\'t match!')
            continue
            
        data['generacja'].append(match.group('generation'))
        data['najlepsza mrówka'].append(match.group('best_ant_num'))
        data['najlepsza wartość'].append(match.group('best_value'))
        data['najgorsza mrówka'].append(match.group('worst_ant_num'))
        data['najgorsza wartość'].append(match.group('worst_value'))
        data['średnia wartość'].append(match.group('mean_value'))
#         data.append([match.group(g) for g in ['generation', 'best_ant_num', 'best_value', 'worst_ant_num', 'worst_value', 'mean_value']])
        
    return data

In [ ]:
def load_parameter_messurements(directory: str) -> pd.DataFrame:
    directory = Path(directory)
    parameter_name = directory.stem
    parameter_dirs = sorted(d for d in directory.iterdir() if d.is_dir())
    
    data = defaultdict(list)
    for parameter_dir in parameter_dirs:
        parameter_value = parameter_dir.name
        parameter_data = load_mrowczenie(parameter_dir / Path('mrowki.txt'))
        for k, v in parameter_data.items():
            data[k].extend(v)
        n = len(parameter_data[list(parameter_data.keys())[0]])
        data['testowany parametr'].extend([parameter_name] * n)
        data['wartość parametru'].extend([parameter_value] * n)
        
    df = pd.DataFrame(data)
    df = df.astype({
        'generacja': int,
        'najlepsza mrówka': int,
        'najlepsza wartość': float,
        'najgorsza mrówka': int,
        'najgorsza wartość': float,
        'średnia wartość': float,
        'wartość parametru': float
    })
    return df

In [ ]:
def plot_single_parameters(df, name) -> go.Figure:
    fig = go.Figure(
        layout=dict(
            xaxis=dict(title='Pokolenie'),
            yaxis=dict(title='Wartość dl'),
            title=f'Średnia ruchoma parametrów dla kolejnych pokoleń {name}',
            margin=dict(t=50, b=0, r=30),
            legend_orientation='h'
        ),
        data=[
            go.Scatter(x=df.generacja, y=df['najlepsza wartość'], name='najlepsza wartość'),
            go.Scatter(x=df.generacja, y=df['najgorsza wartość'].rolling(10, min_periods=5).mean(), name='najgorsza wartość'),
            go.Scatter(x=df.generacja, y=df['średnia wartość'].rolling(5).mean(), name='średnia'),
        ]
    )
    return fig

## aco_ro

In [ ]:
df_aco_ro = load_parameter_messurements("/content/drive/MyDrive/Colab Notebooks/wyniki/aco_ro")
df_aco_ro

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,0,17.689845,38,23.121257,19.466476,aco_ro,0.001
1,1,22,18.664647,45,21.463621,19.287995,aco_ro,0.001
2,2,4,17.429772,20,21.104134,19.319427,aco_ro,0.001
3,3,0,17.429772,47,21.155713,19.450503,aco_ro,0.001
4,4,0,17.429772,17,22.096625,19.404355,aco_ro,0.001
...,...,...,...,...,...,...,...,...
3995,995,0,16.087877,49,22.656364,19.597226,aco_ro,1.000
3996,996,0,16.087877,4,19.635272,19.591294,aco_ro,1.000
3997,997,0,16.087877,13,22.881208,19.530253,aco_ro,1.000
3998,998,0,16.087877,14,22.342904,19.369338,aco_ro,1.000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 0.001

In [ ]:
df_aco_ro_001 = df_aco_ro.query('`wartość parametru` == 0.001').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_ro_001
name = 'aco_ro = 0.001'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
0,0,0,17.689845,38,23.121257,19.466476
1,1,22,18.664647,45,21.463621,19.287995
2,2,4,17.429772,20,21.104134,19.319427
3,3,0,17.429772,47,21.155713,19.450503
4,4,0,17.429772,17,22.096625,19.404355
...,...,...,...,...,...,...
995,995,0,17.259699,31,20.311431,19.189281
996,996,0,17.259699,7,23.992306,19.483301
997,997,0,17.259699,32,20.311431,19.035494
998,998,0,17.259699,10,20.201967,19.375485


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_ro = 0.001'
})


ValueError: ignored

### 0.1

In [ ]:
df_aco_ro_01 = df_aco_ro.query('`wartość parametru` == 0.1').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_ro_01
name = 'aco_ro = 0.1'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
1000,0,0,17.689845,49,20.323809,19.548040
1001,1,0,17.689845,5,20.809588,19.554186
1002,2,0,17.689845,18,20.477887,19.472555
1003,3,0,17.689845,31,23.612748,19.603744
1004,4,45,16.162433,10,22.161019,19.314020
...,...,...,...,...,...,...
1995,995,0,17.500609,11,20.366020,19.311495
1996,996,0,17.500609,13,20.695529,19.167445
1997,997,0,17.500609,25,20.856242,19.232739
1998,998,0,17.500609,36,21.216341,19.390255


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_ro = 0.1'
})


ValueError: ignored

### 0.5

In [ ]:
df_aco_ro_02 = df_aco_ro.query('`wartość parametru` == 0.5').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_ro_02
name = 'aco_ro = 0.5'

df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
2000,0,0,17.689845,7,20.472706,19.688025
2001,1,38,17.602379,27,19.773104,19.944079
2002,2,0,17.602379,10,19.659016,19.424572
2003,3,0,17.602379,21,21.264734,19.440206
2004,4,29,17.563822,21,22.801151,19.597006
...,...,...,...,...,...,...
2995,995,0,16.018651,43,21.877949,19.558501
2996,996,0,16.018651,19,21.000389,20.041919
2997,997,0,16.018651,38,20.852059,19.516944
2998,998,0,16.018651,33,22.499262,19.506410


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_ro = 0.5'
})


ValueError: ignored

### 1.0


In [ ]:
df_aco_ro_07 = df_aco_ro.query('`wartość parametru` == 1.0').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_ro_07
name = 'aco_ro = 1.0'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
3000,0,0,17.689845,12,23.740380,19.724118
3001,1,0,17.689845,27,19.511071,19.508139
3002,2,39,17.647008,33,23.306089,19.832182
3003,3,0,17.647008,10,22.321747,20.028954
3004,4,9,17.451025,29,21.659299,19.658018
...,...,...,...,...,...,...
3995,995,0,16.087877,49,22.656364,19.597226
3996,996,0,16.087877,4,19.635272,19.591294
3997,997,0,16.087877,13,22.881208,19.530253
3998,998,0,16.087877,14,22.342904,19.369338


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_ro = 1.0'
})


ValueError: ignored

### Porównanie

In [ ]:
fig = px.line(df_aco_ro, x='generacja', y='najlepsza wartość', color='wartość parametru',
             title='Wartość najlepszego osobnika w zależności od aco_ro')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Wartość najlepszego osobnika w zależności od aco_ro'
})


ValueError: ignored

In [ ]:
fig = px.line(df_aco_ro, x='generacja', y=df_aco_ro['średnia wartość'].rolling(5).mean(), 
              color='wartość parametru',
              title='Średnia wartość w zależności od aco_ro')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia wartość w zależności od aco_ro'
})


ValueError: ignored

## aco_alfa

In [54]:
df_aco_alfa = load_parameter_messurements("/content/drive/MyDrive/Colab Notebooks/wyniki/aco_alfa")
df_aco_alfa

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,45,16.750883,49,21.968958,19.604503,aco_alfa,0.5
1,1,0,16.750883,8,23.613458,19.799045,aco_alfa,0.5
2,2,0,16.750883,16,21.257485,19.575276,aco_alfa,0.5
3,3,47,17.369803,34,22.419693,19.596351,aco_alfa,0.5
4,4,0,17.369803,41,21.898665,19.749420,aco_alfa,0.5
...,...,...,...,...,...,...,...,...
2995,995,0,17.654196,47,19.589807,18.676697,aco_alfa,2.0
2996,996,0,17.654196,11,19.413171,18.660359,aco_alfa,2.0
2997,997,0,17.654196,5,21.739988,18.610928,aco_alfa,2.0
2998,998,0,17.654196,41,19.589807,18.380930,aco_alfa,2.0


### 0.5

In [43]:
df_aco_alfa_05 = df_aco_alfa[df_aco_alfa['wartość parametru'] == 0.5].drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_alfa_05
name = 'aco_alfa = 0.5'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
0,0,45,16.750883,49,21.968958,19.604503
1,1,0,16.750883,8,23.613458,19.799045
2,2,0,16.750883,16,21.257485,19.575276
3,3,47,17.369803,34,22.419693,19.596351
4,4,0,17.369803,41,21.898665,19.749420
...,...,...,...,...,...,...
995,995,0,19.127595,22,20.905448,19.131230
996,996,0,19.127595,46,23.500579,19.585200
997,997,0,19.127595,33,23.500579,19.546505
998,998,0,19.127595,5,22.171106,19.241195


In [55]:
fig = px.line(df, x="generacja", y="średnia wartość", title='Średnia wartość w zależności od aco_alfa')
fig.update_layout(margin=dict(t=50, b=0))

fig.show()
fig.write_image('fig.svg')

ValueError: ignored

In [45]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_alfa = 0.5'
})


ValueError: ignored

### 1.0

In [46]:
df_aco_alfa_10 = df_aco_alfa.query('`wartość parametru` == 1.0').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_alfa = 1.0'
df = df_aco_alfa_10
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
1000,0,0,17.689845,13,21.632799,19.641927
1001,1,0,17.689845,39,21.407642,19.642451
1002,2,0,17.689845,32,22.434226,19.515814
1003,3,0,17.689845,27,22.351598,19.483345
1004,4,7,17.104989,18,21.916669,19.431419
...,...,...,...,...,...,...
1995,995,0,16.721228,23,20.802080,18.892829
1996,996,0,16.721228,30,21.779255,19.259180
1997,997,0,16.721228,23,19.864803,19.425622
1998,998,0,16.721228,2,19.016135,19.106130


In [47]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_alfa = 1.0'
})


ValueError: ignored

### 2.0

In [48]:
df_aco_alfa_15 = df_aco_alfa.query('`wartość parametru` == 2.0').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_alfa = 2.0'
df = df_aco_alfa_15
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
2000,0,0,17.689845,37,21.668315,19.711217
2001,1,0,17.689845,30,21.283011,19.322335
2002,2,2,18.196735,38,22.000488,19.456536
2003,3,35,16.877477,26,21.690811,19.571461
2004,4,0,16.877477,38,20.845922,19.273594
...,...,...,...,...,...,...
2995,995,0,17.654196,47,19.589807,18.676697
2996,996,0,17.654196,11,19.413171,18.660359
2997,997,0,17.654196,5,21.739988,18.610928
2998,998,0,17.654196,41,19.589807,18.380930


In [49]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_alfa = 2.0'
})


ValueError: ignored

### Porównanie

In [50]:
fig = px.line(df_aco_alfa, x='generacja', y='najlepsza wartość', color='wartość parametru',
             title='Wartość najlepszego osobnika w zależności od aco_alfa')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Wartość najlepszego osobnika w zależności od aco_alfa'
})


ValueError: ignored

In [56]:
fig = px.line(df_aco_alfa, x='generacja', y=df_aco_alfa['średnia wartość'].rolling(5).mean(), 
              color='wartość parametru',
              title='Średnia wartość w zależności od aco_alfa')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia wartość w zależności od aco_alfa'
})


ValueError: ignored

## aco_beta

In [58]:
df_aco_beta = load_parameter_messurements("/content/drive/MyDrive/Colab Notebooks/wyniki/aco_beta")
df_aco_beta

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,37,17.351268,40,23.609978,20.533809,aco_beta,0.5
1,1,0,17.351268,40,21.992931,19.991592,aco_beta,0.5
2,2,0,17.351268,7,22.045917,19.694616,aco_beta,0.5
3,3,0,17.351268,9,21.585926,19.830484,aco_beta,0.5
4,4,0,17.351268,38,22.320802,19.512387,aco_beta,0.5
...,...,...,...,...,...,...,...,...
2995,995,0,16.558809,47,20.688258,19.441203,aco_beta,1.5
2996,996,0,16.558809,36,20.688258,19.516607,aco_beta,1.5
2997,997,0,16.558809,24,19.609097,19.387634,aco_beta,1.5
2998,998,0,16.558809,39,23.905493,19.534728,aco_beta,1.5


### 0.5

In [59]:
df_aco_beta_05 = df_aco_beta.query('`wartość parametru` == 0.5').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_beta_05
name = 'aco_beta = 0.5'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
0,0,37,17.351268,40,23.609978,20.533809
1,1,0,17.351268,40,21.992931,19.991592
2,2,0,17.351268,7,22.045917,19.694616
3,3,0,17.351268,9,21.585926,19.830484
4,4,0,17.351268,38,22.320802,19.512387
...,...,...,...,...,...,...
995,995,0,16.091519,19,22.191386,19.313689
996,996,0,16.091519,47,22.388134,19.572329
997,997,0,16.091519,5,20.756149,19.081004
998,998,0,16.091519,42,22.842603,19.388800


In [60]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_beta = 0.5'
})


ValueError: ignored

### 1.0

In [61]:
df_aco_beta_10 = df_aco_beta.query('`wartość parametru` == 1.0').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_beta = 1.0'
df = df_aco_beta_10
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
1000,0,0,17.204587,45,21.599970,19.873053
1001,1,0,17.204587,46,23.803998,19.933886
1002,2,0,17.204587,1,20.940972,19.565869
1003,3,0,17.204587,35,23.652345,19.454127
1004,4,0,17.204587,33,19.298243,19.367589
...,...,...,...,...,...,...
1995,995,0,17.099374,44,19.054103,17.960631
1996,996,0,17.099374,1,20.344629,17.980218
1997,997,0,17.099374,19,19.029771,17.737257
1998,998,0,17.099374,40,20.647424,17.905069


In [62]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_beta = 1.0'
})


ValueError: ignored

### 1.5

In [63]:
df_aco_beta_20 = df_aco_beta.query('`wartość parametru` == 1.5').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_beta = 1.5'
df = df_aco_beta_20
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
2000,0,0,18.975165,41,20.158753,19.806997
2001,1,0,18.975165,2,21.272711,19.722848
2002,2,9,16.598669,3,22.006206,19.599873
2003,3,0,16.598669,5,22.161980,19.783729
2004,4,0,16.598669,13,21.407972,19.857520
...,...,...,...,...,...,...
2995,995,0,16.558809,47,20.688258,19.441203
2996,996,0,16.558809,36,20.688258,19.516607
2997,997,0,16.558809,24,19.609097,19.387634
2998,998,0,16.558809,39,23.905493,19.534728


In [64]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_beta = 1.5'
})


ValueError: ignored

### Porównanie

In [65]:
fig = px.line(df_aco_beta, x='generacja', y='najlepsza wartość', color='wartość parametru',
             title='Wartość najlepszego osobnika w zależności od aco_beta')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Wartość najlepszego osobnika w zależności od aco_beta'
})


ValueError: ignored

In [66]:
fig = px.line(df_aco_beta, x='generacja', y=df_aco_beta['średnia wartość'].rolling(5).mean(), 
              color='wartość parametru',
              title='Średnia wartość w zależności od aco_beta')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia wartość w zależności od aco_beta'
})


ValueError: ignored

## aco_wsp_ob

In [80]:
df_aco_wsp_ob = load_parameter_messurements("/content/drive/MyDrive/Colab Notebooks/wyniki/aco_wsp_ob")
df_aco_wsp_ob

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,0,17.689845,40,21.095260,19.742251,aco_wsp_ob,0.5
1,1,0,17.689845,37,22.381271,19.694582,aco_wsp_ob,0.5
2,2,0,17.689845,35,21.849675,19.417902,aco_wsp_ob,0.5
3,3,0,17.689845,43,19.228574,19.672569,aco_wsp_ob,0.5
4,4,0,17.689845,28,21.507733,19.538816,aco_wsp_ob,0.5
...,...,...,...,...,...,...,...,...
3995,995,0,16.306861,26,20.608632,19.487021,aco_wsp_ob,2.5
3996,996,0,16.306861,45,22.941470,19.575828,aco_wsp_ob,2.5
3997,997,0,16.306861,17,21.814806,19.332765,aco_wsp_ob,2.5
3998,998,0,16.306861,19,21.385098,19.538604,aco_wsp_ob,2.5


### 0.5

In [81]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 0.5').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_wsp_ob = 0.5'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
0,0,0,17.689845,40,21.095260,19.742251
1,1,0,17.689845,37,22.381271,19.694582
2,2,0,17.689845,35,21.849675,19.417902
3,3,0,17.689845,43,19.228574,19.672569
4,4,0,17.689845,28,21.507733,19.538816
...,...,...,...,...,...,...
995,995,0,16.502761,16,20.901204,19.511876
996,996,0,16.502761,20,20.988729,19.437205
997,997,0,16.502761,29,20.060062,19.215233
998,998,0,16.502761,28,22.891258,19.616187


In [82]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_wsp_ob = 0.5'
})


ValueError: ignored

### 1.5

In [ ]:
df_aco_wsp_ob_10 = df_aco_wsp_ob.query('`wartość parametru` == 1.5').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_wsp_ob = 1.5'
df = df_aco_wsp_ob_10
df

In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

### 2.0

In [ ]:
df_aco_wsp_ob_25 = df_aco_wsp_ob.query('`wartość parametru` == 2.0').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_wsp_ob = 2.0'
df = df_aco_wsp_ob_25
df

In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

### 2.5

In [ ]:
df_aco_wsp_ob_25 = df_aco_wsp_ob.query('`wartość parametru` == 2.5').drop(['testowany parametr', 'wartość parametru'], axis=1)
name = 'aco_wsp_ob = 2.5'
df = df_aco_wsp_ob_25
df

In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

## aco_l_os

In [85]:
df_aco_wsp_ob = load_parameter_messurements("/content/drive/MyDrive/Colab Notebooks/wyniki/aco_l_os")
df_aco_wsp_ob

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,1,18.649940,2,19.922438,18.955399,aco_l_os,10.0
1,1,0,18.649940,6,21.474655,19.640239,aco_l_os,10.0
2,2,2,18.724188,6,21.643722,19.252027,aco_l_os,10.0
3,3,0,18.724188,9,21.825251,19.794109,aco_l_os,10.0
4,4,0,18.724188,6,22.460996,19.865527,aco_l_os,10.0
...,...,...,...,...,...,...,...,...
7995,995,0,16.669637,47,20.583294,18.809256,aco_l_os,50.0
7996,996,0,16.669637,17,18.973309,18.692418,aco_l_os,50.0
7997,997,0,16.669637,41,19.816390,18.548957,aco_l_os,50.0
7998,998,0,16.669637,26,20.093224,18.670953,aco_l_os,50.0


### 200

In [88]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 200').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 200'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
3000,0,0,16.404837,33,21.758709,19.642517
3001,1,0,16.404837,15,22.794161,19.390767
3002,2,0,16.404837,92,23.574654,19.398476
3003,3,0,16.404837,193,21.623761,19.359393
3004,4,0,16.404837,99,22.587116,19.292088
...,...,...,...,...,...,...
3995,995,0,16.404837,107,23.391717,20.648827
3996,996,0,16.404837,139,21.379761,20.579808
3997,997,0,16.404837,56,23.611766,20.690044
3998,998,0,16.404837,23,24.358655,20.661488


In [89]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 200'
})


ValueError: ignored

### 150

In [90]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 150').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 150'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
2000,0,0,16.404837,44,22.707665,19.490523
2001,1,6,18.170726,75,23.816669,19.819551
2002,2,29,17.430323,69,21.427700,19.638430
2003,3,0,17.430323,23,21.229668,19.363836
2004,4,0,17.430323,141,23.751312,19.466172
...,...,...,...,...,...,...
2995,995,0,17.278505,8,19.612736,19.429613
2996,996,0,17.278505,75,20.475031,19.670348
2997,997,0,17.278505,35,20.475031,19.501066
2998,998,0,17.278505,3,22.995646,19.736219


In [91]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 150'
})


ValueError: ignored

### 100

In [92]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 100').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 100'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
1000,0,63,17.113481,92,23.395597,19.578109
1001,1,0,17.113481,67,21.968346,19.546608
1002,2,0,17.113481,94,21.968966,19.576356
1003,3,0,17.113481,96,21.152658,19.578510
1004,4,0,17.113481,78,22.716881,19.620197
...,...,...,...,...,...,...
1995,995,0,16.662329,72,22.586620,19.721274
1996,996,0,16.662329,57,19.981479,19.494739
1997,997,0,16.662329,61,20.964883,19.835059
1998,998,0,16.662329,60,20.317956,19.295975


In [93]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 100'
})


ValueError: ignored

### 50

In [94]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 50').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 500'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
7000,0,48,16.544032,8,22.189296,19.214272
7001,1,0,16.544032,33,21.698865,19.594696
7002,2,8,16.780851,18,21.828245,19.360591
7003,3,0,16.780851,44,22.000488,19.533207
7004,4,0,16.780851,17,19.585089,19.306211
...,...,...,...,...,...,...
7995,995,0,16.669637,47,20.583294,18.809256
7996,996,0,16.669637,17,18.973309,18.692418
7997,997,0,16.669637,41,19.816390,18.548957
7998,998,0,16.669637,26,20.093224,18.670953


In [95]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 500'
})


ValueError: ignored

### 25

In [96]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 25').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 25'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
4000,0,24,16.206226,3,21.877754,19.544947
4001,1,0,16.206226,13,20.411145,19.280698
4002,2,0,16.206226,2,21.543619,19.221951
4003,3,0,16.206226,24,21.101887,19.111211
4004,4,22,19.658476,11,21.762591,19.316060
...,...,...,...,...,...,...
4995,995,0,15.371616,6,20.176345,18.748797
4996,996,0,15.371616,15,19.526018,18.500586
4997,997,0,15.371616,19,20.235321,18.454775
4998,998,0,15.371616,12,20.260018,18.741573


In [97]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 25'
})


ValueError: ignored

### 5

In [98]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 5').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 5'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
6000,0,0,17.628300,4,21.672620,19.650833
6001,1,0,17.628300,1,20.208469,19.823416
6002,2,3,18.480580,4,20.785650,19.102322
6003,3,3,18.642453,2,21.212252,19.179943
6004,4,0,18.642453,3,20.697602,20.173547
...,...,...,...,...,...,...
6995,995,0,15.435156,4,17.757601,18.105907
6996,996,0,15.435156,4,19.613742,18.287873
6997,997,0,15.435156,2,20.850703,18.838052
6998,998,0,15.435156,3,19.385677,18.151638


In [99]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 5'
})


ValueError: ignored

### 3

In [100]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 3').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 3'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
5000,0,0,17.628300,1,21.487929,20.130268
5001,1,1,17.633684,2,19.915464,18.392482
5002,2,2,18.465104,1,22.673056,19.590615
5003,3,1,17.144220,2,18.897532,18.168952
5004,4,0,17.144220,2,22.050021,18.990466
...,...,...,...,...,...,...
5995,995,0,15.896053,1,17.605368,16.787792
5996,996,0,15.896053,2,18.320177,17.451788
5997,997,0,15.896053,2,18.686289,17.627524
5998,998,0,15.896053,2,18.391200,17.016608


In [101]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 3'
})


ValueError: ignored

### Porównanie

In [102]:
fig = px.line(df_aco_wsp_ob, x='generacja', y='najlepsza wartość', color='wartość parametru',
             title='Wartość najlepszego osobnika w zależności od liczby osobników')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Wartość najlepszego osobnika w zależności od liczby osobników'
})


ValueError: ignored

In [104]:
fig = px.line(df_aco_wsp_ob, x='generacja', y=df_aco_wsp_ob['średnia wartość'].rolling(5).mean(), 
              color='wartość parametru',
              title='Średnia wartość w zależności od liczby osobników')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia wartość w zależności od liczby osobników'
})


ValueError: ignored

## aco_l_p

In [105]:
df_aco_wsp_ob = load_parameter_messurements("/content/drive/MyDrive/Colab Notebooks/wyniki/aco_l_p")
df_aco_wsp_ob

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość,testowany parametr,wartość parametru
0,0,48,16.544032,8,22.189296,19.214272,aco_l_p,100.0
1,1,0,16.544032,33,21.698865,19.594696,aco_l_p,100.0
2,2,8,16.780851,18,21.828245,19.360591,aco_l_p,100.0
3,3,0,16.780851,44,22.000488,19.533207,aco_l_p,100.0
4,4,0,16.780851,17,19.585089,19.306211,aco_l_p,100.0
...,...,...,...,...,...,...,...,...
4095,495,0,16.669637,32,20.211334,18.420012,aco_l_p,500.0
4096,496,0,16.669637,43,20.211334,18.434149,aco_l_p,500.0
4097,497,0,16.669637,30,19.255982,18.322538,aco_l_p,500.0
4098,498,0,16.669637,40,19.694181,18.566778,aco_l_p,500.0


### 2000

In [106]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 2000').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 2000'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
1600,0,48,16.544032,8,22.189296,19.214272
1601,1,0,16.544032,33,21.698865,19.594696
1602,2,8,16.780851,18,21.828245,19.360591
1603,3,0,16.780851,44,22.000488,19.533207
1604,4,0,16.780851,17,19.585089,19.306211
...,...,...,...,...,...,...
3595,1995,0,16.669637,15,19.667754,18.923191
3596,1996,0,16.669637,22,19.659607,18.814810
3597,1997,0,16.669637,17,19.525379,18.986826
3598,1998,0,16.669637,20,19.444776,18.903757


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 200'
})


ValueError: ignored

### 150

In [ ]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 150').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 150'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
2000,0,0,16.404837,44,22.707665,19.490523
2001,1,6,18.170726,75,23.816669,19.819551
2002,2,29,17.430323,69,21.427700,19.638430
2003,3,0,17.430323,23,21.229668,19.363836
2004,4,0,17.430323,141,23.751312,19.466172
...,...,...,...,...,...,...
2995,995,0,17.278505,8,19.612736,19.429613
2996,996,0,17.278505,75,20.475031,19.670348
2997,997,0,17.278505,35,20.475031,19.501066
2998,998,0,17.278505,3,22.995646,19.736219


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 150'
})


ValueError: ignored

### 100

In [ ]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 100').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 100'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
1000,0,63,17.113481,92,23.395597,19.578109
1001,1,0,17.113481,67,21.968346,19.546608
1002,2,0,17.113481,94,21.968966,19.576356
1003,3,0,17.113481,96,21.152658,19.578510
1004,4,0,17.113481,78,22.716881,19.620197
...,...,...,...,...,...,...
1995,995,0,16.662329,72,22.586620,19.721274
1996,996,0,16.662329,57,19.981479,19.494739
1997,997,0,16.662329,61,20.964883,19.835059
1998,998,0,16.662329,60,20.317956,19.295975


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 100'
})


ValueError: ignored

### 50

In [ ]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 50').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 500'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
7000,0,48,16.544032,8,22.189296,19.214272
7001,1,0,16.544032,33,21.698865,19.594696
7002,2,8,16.780851,18,21.828245,19.360591
7003,3,0,16.780851,44,22.000488,19.533207
7004,4,0,16.780851,17,19.585089,19.306211
...,...,...,...,...,...,...
7995,995,0,16.669637,47,20.583294,18.809256
7996,996,0,16.669637,17,18.973309,18.692418
7997,997,0,16.669637,41,19.816390,18.548957
7998,998,0,16.669637,26,20.093224,18.670953


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 500'
})


ValueError: ignored

### 25

In [ ]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 25').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 25'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
4000,0,24,16.206226,3,21.877754,19.544947
4001,1,0,16.206226,13,20.411145,19.280698
4002,2,0,16.206226,2,21.543619,19.221951
4003,3,0,16.206226,24,21.101887,19.111211
4004,4,22,19.658476,11,21.762591,19.316060
...,...,...,...,...,...,...
4995,995,0,15.371616,6,20.176345,18.748797
4996,996,0,15.371616,15,19.526018,18.500586
4997,997,0,15.371616,19,20.235321,18.454775
4998,998,0,15.371616,12,20.260018,18.741573


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 25'
})


ValueError: ignored

### 5

In [ ]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 5').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 5'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
6000,0,0,17.628300,4,21.672620,19.650833
6001,1,0,17.628300,1,20.208469,19.823416
6002,2,3,18.480580,4,20.785650,19.102322
6003,3,3,18.642453,2,21.212252,19.179943
6004,4,0,18.642453,3,20.697602,20.173547
...,...,...,...,...,...,...
6995,995,0,15.435156,4,17.757601,18.105907
6996,996,0,15.435156,4,19.613742,18.287873
6997,997,0,15.435156,2,20.850703,18.838052
6998,998,0,15.435156,3,19.385677,18.151638


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 5'
})


ValueError: ignored

### 3

In [ ]:
df_aco_wsp_ob_05 = df_aco_wsp_ob.query('`wartość parametru` == 3').drop(['testowany parametr', 'wartość parametru'], axis=1)
df = df_aco_wsp_ob_05
name = 'aco_l_os = 3'
df

,generacja,najlepsza mrówka,najlepsza wartość,najgorsza mrówka,najgorsza wartość,średnia wartość
5000,0,0,17.628300,1,21.487929,20.130268
5001,1,1,17.633684,2,19.915464,18.392482
5002,2,2,18.465104,1,22.673056,19.590615
5003,3,1,17.144220,2,18.897532,18.168952
5004,4,0,17.144220,2,22.050021,18.990466
...,...,...,...,...,...,...
5995,995,0,15.896053,1,17.605368,16.787792
5996,996,0,15.896053,2,18.320177,17.451788
5997,997,0,15.896053,2,18.686289,17.627524
5998,998,0,15.896053,2,18.391200,17.016608


In [ ]:
fig = plot_single_parameters(df, name)

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia ruchoma parametrów dla kolejnych pokoleń aco_l_os = 3'
})


ValueError: ignored

### Porównanie

In [ ]:
fig = px.line(df_aco_wsp_ob, x='generacja', y='najlepsza wartość', color='wartość parametru',
             title='Wartość najlepszego osobnika w zależności od liczby osobników')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Wartość najlepszego osobnika w zależności od liczby osobników'
})


ValueError: ignored

In [ ]:
fig = px.line(df_aco_wsp_ob, x='generacja', y=df_aco_wsp_ob['średnia wartość'].rolling(5).mean(), 
              color='wartość parametru',
              title='Średnia wartość w zależności od liczby osobników')
fig.update_yaxes(title_text='Wartość dl')
fig.update_xaxes(title_text='Pokolenie')
fig.update_layout(margin=dict(t=50, b=0, r=30), legend_orientation='v')

print(fig.layout.title)
fig.show()
fig.write_image('fig.svg', width=1000)

layout.Title({
    'text': 'Średnia wartość w zależności od liczby osobników'
})


ValueError: ignored